# Working with geospatial data
We now turn to handling special data types, including geo-spatial data. This is a bit different than most data types we've handled. 
1. The process of determining Latitude and Longitude from a specific human description is called "geocoding". 
2. Geocoding is data intensive. It's not done on your computer; the databases are too large to fit. 
3. Instead, there are Application Programming Interfaces that allow you to translate between human concepts and the more abstract concepts of Latitude and Longitude. 
4. Most geocoding solutions are not free. You can use them as a developer with so-called "rate limiting". This allows you to submit a limited number of requests per hour for free. You would need to subscribe and pay a monthly fee to use them at high rates, e.g., to drive a web application or geocode a large repository of addresses. 
5. Depending upon the geocoding solution, you might need an "API key" to use it. This is a string, similar to a password, that identifies you and your application uniquely and allows you to use the API within rate limits. 

Let's play with this concept a bit and solve some simple problems.  

First, note that you'll have to install geopy for this to work. In a Jupyter terminal, type 
```
pip install geopy
pip install folium
pip install polyline
```
before starting these problems. 

Note that in this problem, running the solution directly can break folium! Thus we save the solution to an HTML file and zip it with the solution code.

*Question 1*: Read https://towardsdatascience.com/things-to-do-with-latitude-longitude-data-using-geopy-python-1d356ed1ae30 and then utilize geopy to calculate the latitude and longitude of Halligan Hall (161 College Avenue, Medford, MA 01801). Put these into the object `halligan`. Then read 
https://python-visualization.github.io/folium/quickstart.html and create an interactive map centered on Halligan Hall, with a marker labeled "Halligan Hall". Choose `zoom_start` reasonably to make the map depict the Tufts campus. 

Note: You will get the location of the pool. This is true in any geocoder. That's because there are two "161 College Avenue" addresses! 

In [2]:
import folium
import geopy
# BEGIN SOLUTION
from geopy.geocoders import Nominatim
geocoder = Nominatim(user_agent = 'lab_geo_app')

address = '161 College Avenue, Medford, MA 01801'

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geocoder.geocode, min_delay_seconds = 1,   return_value_on_exception = None) 
# adding 1 second padding between calls

# after initiating geocoder
location = geocode(address)
# returns location object with longitude, latitude and altitude instances
(location.latitude, location.longitude)

halligan = location

m = folium.Map(location=(halligan.latitude, halligan.longitude), zoom_start=15)
folium.Marker(
    location=[halligan.latitude, halligan.longitude],
    tooltip="Halligan Hall",
    popup="Halligan Hall",
    icon=folium.Icon(icon="cloud"),
).add_to(m)
m
# END SOLUTION

*Question 2:* Read https://api-v3.mbta.com/ and then study the output of the following query to the MBTA database, which obtains the locations -- in real time -- of _every bus in the MBTA._  Use this output to plot the location of every bus in the data and label it with its vehicle number and routes. 

In [3]:
import requests
url = 'https://api-v3.mbta.com/vehicles/'
data = requests.get(url).json()
from pprint import pprint
pprint(data)

{'data': [{'attributes': {'bearing': 353,
                          'carriages': [],
                          'current_status': 'STOPPED_AT',
                          'current_stop_sequence': 24,
                          'direction_id': 1,
                          'label': '1772',
                          'latitude': 42.328125339,
                          'longitude': -71.08318608,
                          'occupancy_status': 'FULL',
                          'revenue': 'REVENUE',
                          'speed': None,
                          'updated_at': '2024-10-28T14:06:54-04:00'},
           'id': 'y1772',
           'links': {'self': '/vehicles/y1772'},
           'relationships': {'route': {'data': {'id': '23', 'type': 'route'}},
                             'stop': {'data': {'id': '21151', 'type': 'stop'}},
                             'trip': {'data': {'id': '64462825',
                                               'type': 'trip'}}},
           'type': 'vehicle'},


In [4]:
# BEGIN SOLUTION
import folium
import requests

# Centered in Boston
m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

for bus in data['data']:
    attributes = bus['attributes']
    
    # Extract latitude, longitude, vehicle label, and route ID
    latitude = attributes['latitude']
    longitude = attributes['longitude']
    vehicle_label = attributes['label']
    route_id = bus['relationships']['route']['data']['id'] if 'route' in bus['relationships'] else 'N/A'
    
    # Create popup text with vehicle and route info
    popup_text = f"Vehicle: {vehicle_label}<br>Route: {route_id}"
    
    # Add a marker for each bus
    folium.Marker(
        [latitude, longitude],
        popup=popup_text,
        tooltip=vehicle_label,  # Shows vehicle label on hover
        icon=folium.Icon(color="blue", icon="bus", prefix="fa")
    ).add_to(m)

m
# END SOLUTION

*Question 3:* Study the output of the query below and then visualize the route of bus route 96 on a folium map. This requires converting the routes to shapes that folium can understand. 

*Hint:* the shapes are encoded and you'll need the google-based polyline format decoder described here: https://polyline.readthedocs.io/en/v1.1/ Use the instructions for *decoding.* 

In [5]:
# the documentation is rather opaque on how to do this: 
url = 'https://api-v3.mbta.com/shapes?filter[route]=96'
shapes = requests.get(url).json()
pprint(shapes)

{'data': [{'attributes': {'polyline': 'eesaGdmaqL]K_@Gk@Ck@A[?]BU?sAHg@DWDODKJMPM\\KNEQAOU`@c@`@EBC@[PeCn@g@HM@??[@cBG_BKmBIq@Eq@GyAGoAGcDQ??OAiDQoFYcCO_AEM?OA??yAIyAIeBK}AK_ACy@FiAPu@N??YFa@FSDaARw@\\UL[T??WT_B`Bm@r@e@n@eCgEs@iA_AaBSR[X??MJ{AtAwAnAIH}@j@eB|@??_@Rs@u@m@k@WY{@}@eBiB{ApEgAk@??c@Uo@jDI|AQrA??Kt@eAg@_@Q_Ae@uAs@g@WYOY]Yc@Q_@Sa@cAaDQk@??Oc@i@wAwAiCoCiE????{@kAGm@W_@OIOAEBEFCHCH[^YZ]B??}@HiCN_Gf@gDVeAJ??m@Di@Fc@CA?aAc@cBkAKE??OICAe@Ga@H{A`AqA|@iAz@}@v@MJ??s@p@y@|@{@bAsBpCeAjBeArBgArBg@x@??KPwA{@UM}@g@_@WqAq@UM??qAw@{BqA_@ULe@??h@sBTk@dBeGf@iBt@aC??HS~@yCVeApAaEl@qB??FOBq@Cs@OgDS{CW{C??AW{@J}@F_ABw@?e@?a@Ag@C??OAyBMs@Ck@?K[IQEMOHa@JSDM?}@???oCAU?}@Kk@KSGwAu@e@]WSIMEOASb@_CRiA??RgAn@uCRaA`DjBp@aCBu@C[EUYe@??wAqA{@|CuAM'},
           'id': '960145',
           'links': {'self': '/shapes/960145'},
           'type': 'shape'},
          {'attributes': {'polyline': 'su{aG|~~pLc@EgCI@vBFvC??@V?j@PhBLlBXzAZ`@RRPJ??TLpAt@r@R`@Jn@FL@~@@hCA??P?jBBj@?r@BxBLd@B??P@`@@d@?v@?~@C|@Gz@KDf@??R

In [20]:
# BEGIN SOLUTION
from polyline.codec import PolylineCodec

m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

for shape in shapes['data']:
    encoded_string = shape['attributes']['polyline']
    coordinates = PolylineCodec().decode(encoded_string)

    trail_coordinates = []
    trail_coordinates.append(coordinates)
    
    folium.PolyLine(trail_coordinates, tooltip="Coast").add_to(m)

m



# END SOLUTION